# Механізми уваги та трансформери

Одним із головних недоліків рекурентних мереж є те, що всі слова в послідовності мають однаковий вплив на результат. Це призводить до субоптимальної продуктивності стандартних моделей LSTM-енкодер-декодер для задач послідовність-у-послідовність, таких як розпізнавання іменованих сутностей та машинний переклад. Насправді, окремі слова у вхідній послідовності часто мають більший вплив на вихідні результати, ніж інші.

Розглянемо модель послідовність-у-послідовність, наприклад, машинний переклад. Вона реалізується за допомогою двох рекурентних мереж, де одна мережа (**енкодер**) стискає вхідну послідовність у прихований стан, а інша, **декодер**, розгортає цей прихований стан у перекладений результат. Проблема цього підходу полягає в тому, що фінальному стану мережі важко запам'ятати початок речення, що призводить до низької якості моделі для довгих речень.

**Механізми уваги** забезпечують спосіб зважування контекстуального впливу кожного вхідного вектора на кожне передбачення виходу RNN. Це реалізується шляхом створення "ярликів" між проміжними станами вхідної RNN та вихідної RNN. Таким чином, при генерації вихідного символу $y_t$ ми враховуємо всі приховані стани входу $h_i$ з різними ваговими коефіцієнтами $\alpha_{t,i}$.

![Зображення моделі енкодер/декодер з додатковим шаром уваги](../../../../../translated_images/uk/encoder-decoder-attention.7a726296894fb567.webp)
*Модель енкодер-декодер з механізмом додаткової уваги у [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитовано з [цього блогу](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матриця уваги $\{\alpha_{i,j}\}$ представляє ступінь, до якого певні вхідні слова впливають на генерацію конкретного слова у вихідній послідовності. Нижче наведено приклад такої матриці:

![Зображення прикладу вирівнювання, знайденого RNNsearch-50, взято з Bahdanau - arviz.org](../../../../../translated_images/uk/bahdanau-fig3.09ba2d37f202a6af.webp)

*Зображення взято з [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Рис.3)*

Механізми уваги є основою багатьох сучасних або близьких до сучасних досягнень у галузі обробки природної мови. Однак додавання уваги значно збільшує кількість параметрів моделі, що призводить до проблем масштабування з RNN. Ключовим обмеженням масштабування RNN є те, що рекурентна природа моделей ускладнює пакетну обробку та паралелізацію навчання. У RNN кожен елемент послідовності потрібно обробляти в послідовному порядку, що унеможливлює легку паралелізацію.

Використання механізмів уваги разом із цим обмеженням призвело до створення сучасних трансформерних моделей, які ми сьогодні знаємо та використовуємо, таких як BERT і OpenGPT3.

## Трансформерні моделі

Замість передачі контексту кожного попереднього передбачення на наступний етап оцінки, **трансформерні моделі** використовують **позиційні кодування** та **увагу**, щоб захопити контекст заданого входу в межах наданого текстового вікна. Зображення нижче показує, як позиційні кодування разом із увагою можуть захоплювати контекст у межах заданого вікна.

![Анімований GIF, що показує, як виконуються оцінки в трансформерних моделях.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Оскільки кожна вхідна позиція незалежно відображається на кожну вихідну позицію, трансформери можуть краще паралелізуватися, ніж RNN, що дозволяє створювати значно більші та виразніші мовні моделі. Кожна голова уваги може використовуватися для вивчення різних взаємозв’язків між словами, що покращує виконання задач обробки природної мови.

## Створення простої трансформерної моделі

Keras не містить вбудованого шару трансформера, але ми можемо створити його самостійно. Як і раніше, ми зосередимося на класифікації тексту з використанням набору даних AG News, але варто зазначити, що трансформерні моделі демонструють найкращі результати у складніших задачах обробки природної мови.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Нові шари в Keras повинні успадковувати клас `Layer` і реалізовувати метод `call`. Почнемо з шару **Positional Embedding**. Ми використаємо [деякий код з офіційної документації Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Ми припустимо, що всі вхідні послідовності заповнені до довжини `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Цей шар складається з двох шарів `Embedding`: для вбудовування токенів (у спосіб, який ми обговорювали раніше) і позицій токенів. Позиції токенів створюються як послідовність натуральних чисел від 0 до `maxlen` за допомогою `tf.range`, а потім передаються через шар вбудовування. Два отримані вектори вбудовування додаються, утворюючи позиційно-вбудоване представлення вхідних даних форми `maxlen`$\times$`embed_dim`.

Тепер давайте реалізуємо блок трансформера. Він прийматиме вихід раніше визначеного шару вбудовування:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Трансформер застосовує `MultiHeadAttention` до вхідних даних із позиційним кодуванням, щоб отримати вектор уваги розмірності `maxlen`$\times$`embed_dim`, який потім змішується з вхідними даними та нормалізується за допомогою `LayerNormalization`.

> **Примітка**: `LayerNormalization` схожа на `BatchNormalization`, яку ми розглядали в розділі *Комп'ютерне бачення* цього навчального курсу, але вона нормалізує виходи попереднього шару для кожного навчального зразка незалежно, приводячи їх до діапазону [-1..1].

Вихід цього шару потім передається через `Dense` мережу (у нашому випадку - двошаровий перцептрон), і результат додається до фінального виходу (який знову проходить нормалізацію).


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Моделі трансформерів BERT

**BERT** (Bidirectional Encoder Representations from Transformers) — це велика багатошарова трансформерна мережа з 12 шарами для *BERT-base* і 24 для *BERT-large*. Модель спочатку проходить попереднє навчання на великому корпусі текстових даних (WikiPedia + книги) за допомогою неконтрольованого навчання (прогнозування замаскованих слів у реченні). Під час попереднього навчання модель засвоює значний рівень розуміння мови, який потім можна використовувати з іншими наборами даних за допомогою тонкого налаштування. Цей процес називається **трансферним навчанням**.

![зображення з http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/uk/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Існує багато варіацій архітектур трансформерів, включаючи BERT, DistilBERT, BigBird, OpenGPT3 та інші, які можна налаштовувати.

Давайте подивимося, як ми можемо використати попередньо навчений BERT для вирішення нашої традиційної задачі класифікації послідовностей. Ми запозичимо ідею та трохи коду з [офіційної документації](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Для завантаження попередньо навчених моделей ми будемо використовувати **Tensorflow hub**. Спочатку завантажимо векторизатор, специфічний для BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Важливо використовувати той самий векторизатор, на якому була навчена початкова мережа. Крім того, векторизатор BERT повертає три компоненти:
* `input_word_ids`, що є послідовністю номерів токенів для вхідного речення
* `input_mask`, який показує, яка частина послідовності містить фактичний вхід, а яка є заповненням. Це схоже на маску, створену шаром `Masking`
* `input_type_ids` використовується для завдань мовного моделювання і дозволяє вказати два вхідні речення в одній послідовності.

Після цього ми можемо створити екстрактор ознак BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Отже, шар BERT повертає кілька корисних результатів:
* `pooled_output` — це результат усереднення всіх токенів у послідовності. Його можна розглядати як інтелектуальне семантичне представлення всієї мережі. Це еквівалент виходу шару `GlobalAveragePooling1D` у нашій попередній моделі.
* `sequence_output` — це вихід останнього шару трансформера (відповідає виходу `TransformerBlock` у нашій моделі вище).
* `encoder_outputs` — це виходи всіх шарів трансформера. Оскільки ми завантажили 4-шарову модель BERT (як ви, мабуть, здогадалися з назви, яка містить `4_H`), вона має 4 тензори. Останній з них збігається з `sequence_output`.

Тепер ми визначимо наскрізну модель класифікації. Ми будемо використовувати *функціональне визначення моделі*, коли ми визначаємо вхід моделі, а потім надаємо серію виразів для обчислення її виходу. Ми також зробимо ваги моделі BERT незмінними (не тренованими) і будемо тренувати лише фінальний класифікатор:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Попри те, що кількість параметрів для навчання невелика, процес досить повільний, оскільки екстрактор ознак BERT є обчислювально важким. Схоже, нам не вдалося досягти прийнятної точності, або через недостатнє навчання, або через недостатню кількість параметрів моделі.

Спробуймо розморозити ваги BERT і навчити його також. Це вимагає дуже маленької швидкості навчання, а також більш обережної стратегії навчання з **warmup**, використовуючи оптимізатор **AdamW**. Ми будемо використовувати пакет `tf-models-official` для створення оптимізатора:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Як ви можете помітити, навчання проходить досить повільно — але ви можете спробувати експериментувати та тренувати модель протягом кількох епох (5-10), щоб побачити, чи вдасться отримати кращий результат у порівнянні з підходами, які ми використовували раніше.

## Бібліотека Huggingface Transformers

Ще один дуже поширений (і трохи простіший) спосіб використання моделей Transformer — це [пакет HuggingFace](https://github.com/huggingface/), який надає прості будівельні блоки для різних завдань обробки природної мови (NLP). Він доступний як для Tensorflow, так і для PyTorch — ще однієї дуже популярної бібліотеки для нейронних мереж.

> **Примітка**: Якщо вам не цікаво побачити, як працює бібліотека Transformers, ви можете пропустити до кінця цього ноутбука, оскільки ви не побачите нічого принципово нового порівняно з тим, що ми робили вище. Ми повторимо ті самі кроки навчання моделі BERT, використовуючи іншу бібліотеку та значно більшу модель. Таким чином, процес включає досить тривале навчання, тому ви можете просто переглянути код.

Давайте подивимося, як можна вирішити нашу задачу за допомогою [Huggingface Transformers](http://huggingface.co).


Перше, що нам потрібно зробити, це обрати модель, яку ми будемо використовувати. Окрім деяких вбудованих моделей, Huggingface має [онлайн-репозиторій моделей](https://huggingface.co/models), де ви можете знайти багато інших попередньо навчених моделей, створених спільнотою. Усі ці моделі можна завантажити та використовувати, просто вказавши назву моделі. Усі необхідні бінарні файли для моделі будуть автоматично завантажені.

У деяких випадках вам може знадобитися завантажити власні моделі. У такому разі ви можете вказати директорію, яка містить усі відповідні файли, включаючи параметри для токенайзера, файл `config.json` із параметрами моделі, бінарні ваги тощо.

З назви моделі ми можемо створити як саму модель, так і токенайзер. Почнемо з токенайзера:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Об'єкт `tokenizer` містить функцію `encode`, яку можна безпосередньо використовувати для кодування тексту:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Ми також можемо використовувати токенізатор для кодування послідовності у спосіб, придатний для передачі моделі, тобто включаючи поля `token_ids`, `input_mask` тощо. Ми також можемо вказати, що хочемо тензори Tensorflow, надавши аргумент `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

У нашому випадку ми будемо використовувати попередньо навчений BERT-модель під назвою `bert-base-uncased`. *Uncased* означає, що модель не чутлива до регістру.

Під час навчання моделі нам потрібно надати токенізовану послідовність як вхідні дані, тому ми створимо конвеєр обробки даних. Оскільки `tokenizer.encode` є функцією Python, ми використаємо той самий підхід, що й у попередньому розділі, викликаючи її за допомогою `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Тепер ми можемо завантажити фактичну модель, використовуючи пакет `BertForSequenceClassfication`. Це гарантує, що наша модель вже має необхідну архітектуру для класифікації, включаючи фінальний класифікатор. Ви побачите попередження, яке повідомляє, що ваги фінального класифікатора не ініціалізовані, і модель потребуватиме попереднього навчання - це абсолютно нормально, адже саме це ми збираємося зробити!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Як видно з `summary()`, модель містить майже 110 мільйонів параметрів! Ймовірно, якщо ми хочемо виконати просте завдання класифікації на відносно невеликому наборі даних, ми не хочемо тренувати базовий шар BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Тепер ми готові розпочати навчання!

> **Примітка**: Навчання повномасштабної моделі BERT може зайняти дуже багато часу! Тому ми будемо тренувати її лише на перших 32 батчах. Це лише для демонстрації налаштування процесу навчання моделі. Якщо ви хочете спробувати повномасштабне навчання – просто видаліть параметри `steps_per_epoch` та `validation_steps`, і приготуйтеся чекати!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Якщо збільшити кількість ітерацій, зачекати достатньо часу та тренувати модель протягом кількох епох, можна очікувати, що класифікація за допомогою BERT забезпечить найкращу точність! Це тому, що BERT уже досить добре розуміє структуру мови, і нам потрібно лише доопрацювати фінальний класифікатор. Однак, через те, що BERT є великою моделлю, увесь процес навчання займає багато часу та потребує значних обчислювальних ресурсів! (GPU, і бажано більше ніж один).

> **Примітка:** У нашому прикладі ми використовували одну з найменших попередньо натренованих моделей BERT. Існують більші моделі, які, ймовірно, дадуть кращі результати.


## Основні висновки

У цьому розділі ми розглянули найсучасніші архітектури моделей, засновані на **трансформерах**. Ми застосували їх для завдання класифікації тексту, але аналогічно моделі BERT можуть використовуватися для вилучення сутностей, відповіді на запитання та інших завдань обробки природної мови.

Моделі на основі трансформерів представляють сучасний рівень розвитку в NLP, і в більшості випадків це має бути перше рішення, з якого ви починаєте експериментувати при впровадженні власних NLP-рішень. Однак розуміння базових принципів рекурентних нейронних мереж, які обговорювалися в цьому модулі, є надзвичайно важливим, якщо ви хочете створювати просунуті нейронні моделі.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
